In [1]:
from traitlets.config.manager import BaseJSONConfigManager
cm = BaseJSONConfigManager()
cm.update('livereveal', {
              'theme': 'sky',
              'transition': 'zoom',
              'start_slideshow_at': 'selected',
            'scroll': True
})

{'scroll': True,
 'start_slideshow_at': 'selected',
 'theme': 'sky',
 'transition': 'zoom'}

# Занятие 27. 

## Системное программирование и сети.


## План занятия

- повторение многопоточности
- клиент-серверные программы

Внутри одного ядра процессора все задачи выполняются **последовательно**. Иллюзия выполнения нескольких задач возникает из-за планировщика задач, который постоянно переключает контекст работы. Ядра процессора работают **параллельно**

#### В чем отличие потоков от процессов?
 У процессов **неразделяемая память**, а у потоков **общая**. Процессы требуют больше памяти.

#### Основные механизмы решения многопоточных задач
- мьютексы 
- семафоры
- условные переменные

- Мьютексы замедляют работу многопоточной программы, но могут быть допустимы, если внутри них содержится малая часть программы. 
- для 4-х ядерного процессора оптимальное количество потоков --- 4

## Cети

### Модель OSI
- Прикладной уровень
- Уровень представления
- Сеансовый уровень
- Транспортный уровень
- Сетевой уровень
- Канальный уровень
- Физический уровень

Подробнее ($\href{https://ru.wikipedia.org/wiki/%D0%A1%D0%B5%D1%82%D0%B5%D0%B2%D0%B0%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C_OSI}{wiki}$)

### Клиент-серверная архитектура

Клиент --- веб-браузер, принимающий запрос от пользователя.

Сервер --- машина, которая отвечает на запросы.

<img src="./pics/client.png" width='50%'>

### Пишем сервер

In [ ]:
import socket

# https://docs.python.org/3/library/socket.html
sock=socket.socket(socket.AF_INET, socket.SOCK_STREAM) # дефолтные параметры
sock.bind(("127.0.0.1", 10002))   # max port 65535
sock.listen(socket.SOMAXCONN)  ##дефолтные параметры

conn, addr = sock.accept()
while True:
    data = conn.recv(1024)
    if not data:
        break
    print(data.decode("utf8"))
    
conn.close()
sock.close()

### Пишем клиента  (в отдельном ноутбуке)

#### Подключаемся к серверу

In [ ]:
import socket

sock = socket.socket()
sock.connect(("127.0.0.1", 10001))

#### Шлем сообщение

In [ ]:
sock.sendall("ping".encode("utf8"))

#### Закрываем соединение вместе с сервером


In [ ]:
sock.close()

После разрыва соедениения снова записать не получится.

#### Вторая версия сервера

In [ ]:
import socket

with socket.socket() as sock:
    sock.bind(("127.0.0.1", 10001))
    sock.listen()
    
    while True:
        conn, addr = sock.accept()
        with conn:
            while True:
                data = conn.recv(1024)
                if not data:
                    break
                print(data.decode("utf8"))

#### Более короткая запись подключения клиента

In [ ]:
sock = socket.create_connection(("127.0.0.1", 10001))
sock.sendall("ping".encode("utf8"))
sock.close()

Теперь с помощью сервера передадим данные клиенту

In [ ]:
import socket

with socket.socket() as sock:
    sock.bind(("127.0.0.1", 10001))
    sock.listen()
    
    while True:
        conn, addr = sock.accept()
        with conn:
            while True:
                data = conn.recv(1024)
                conn.send(b'Hello, I"m a server')
                if not data:
                    break
                print(data.decode("utf8"))

Для того, чтобы принять сообщение добавим метод ``recv`` для клиента

In [ ]:
sock = socket.create_connection(("127.0.0.1", 10001))
sock.sendall("ping".encode("utf8"))
resp = sock.recv(1024)
print(resp)

Если и клиент и сервер ждут действий друг от друга, то происходит взаимная блокировка (deadlock), в результате которой соединение зависает бесконечно долго.

### Таймауты 

Добавим ограничение на промежутки времени между активных действий пользователя ``settimeout``

In [ ]:
# для сервера
import time
import socket

with socket.socket() as sock:
    sock.bind(("", 10002)) 
    sock.listen()
    while True:
#         time.sleep(4)
        conn, addr = sock.accept()
        
        conn.settimeout(3)  # timeout := None|0|gt 0 
        with conn:
            while True:
                try:
                    data = conn.recv(1024)
#                     print(len(data))
                except socket.timeout:
                    print("close connection by timeout")
                    break
                
                if not data:
                    break
                print(data.decode("utf8"))

In [ ]:
# для клиента 
with socket.create_connection(("127.0.0.1", 10002), 3) as sock:
    # set socket read timeout
    sock.settimeout(2)
    try:
        sock.sendall("ping".encode("utf8"))
        resp= sock.recv(1024)
        print(resp)
        resp = sock.recv(1024)
    except socket.timeout:
        print("send data timeout")
    except socket.error as ex:
        print("send data error:", ex)

#### Передача больших данных

In [ ]:
from random import randint

x = [randint(1,100) for i in range(40)]

with socket.create_connection(("127.0.0.1", 10002), 3) as sock:
    # set socket read timeout
    sock.settimeout(2)
    try:
        y= sorted(x)
        for i in y:
            sock.sendall(bytes(i))
    except socket.timeout:
        print("send data timeout")
    except socket.error as ex:
        print("send data error:", ex)


### Одновременная работа нескольких соединений

#### Cервер

In [5]:
import time
import socket 
with socket.socket() as sock:
    sock.bind(("", 10002))
    sock.listen(2)
    while True:
        conn, addr = sock.accept()
        with conn:
            while True:
                try:
                    data = conn.recv(1024)
                except socket.timeout:
                    print("close connection by timeout")
                    break
                
                if not data:
                    break
                print(data.decode("utf8"))

#### Клиент

In [ ]:
import socket 

sock = socket.create_connection(("127.0.0.1", 10001))
sock2 = socket.create_connection(("127.0.0.1", 10001))

In [ ]:
sock.sendall("ping".encode("utf8"))
sock2.sendall("ping2".encode("utf8"))

In [ ]:
sock.close()

В таком режиме обрабатывается только 1 клиент и второй ожидает в очереди.

##### Для каждого клиента будет добавлять новый поток

#### Cервер

In [ ]:
import threading
import socket

def process_request(conn, addr):
    print("connected client:", addr)
    with conn:
        while True:
            data = conn.recv(1024)
            if not data:
                break
            print(data.decode("utf8"))

with socket.socket() as sock:
    sock.bind(("", 10001))
    sock.listen()
    while True:
        conn, addr = sock.accept()
        th = threading.Thread(target=process_request, args=(conn, addr))
        th.start()


#### Клиент

In [ ]:
import socket 

sock = socket.create_connection(("127.0.0.1", 10001))
sock2 = socket.create_connection(("127.0.0.1", 10001))

In [ ]:
sock.sendall("ping".encode("utf8"))
sock2.sendall("ping2".encode("utf8"))

In [ ]:
sock3 = socket.create_connection(("127.0.0.1", 10001))

In [ ]:
sock.sendall("ping".encode("utf8"))
sock2.sendall("ping2".encode("utf8"))
sock3.sendall("ping3".encode("utf8"))

In [2]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()